In [1]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def ch1903_to_latlng(x, y):
    x_aux = (x - 600000) / 1000000
    y_aux = (y - 200000) / 1000000
    lat = 16.9023892 + 3.238272 * y_aux - 0.270978 * x_aux ** 2 - 0.002528 * y_aux ** 2 - 0.0447 * x_aux ** 2 * y_aux - 0.014 * y_aux ** 3
    lng = 2.6779094 + 4.728982 * x_aux + 0.791484 * x_aux * y_aux + 0.1306 * x_aux * y_aux ** 2 - 0.0436 * x_aux ** 3
    lat = (lat * 100) / 36
    lng = (lng * 100) / 36
    return [lat, lng]

### Temperature and Conductivity

In [3]:
df = pd.read_csv("CTD.csv", encoding='latin1', sep=";")
coords = np.array(df["Eastings/Northings"])
lat, lng = np.array([ch1903_to_latlng(float(n), float(d)) for n, d in (pair.split(' / ') for pair in coords)]).T
df["latitude"] = lat
df["longitude"] = lng
df["depth"] = df["Tiefe [m]"]
df['time'] = pd.to_datetime(df['Sampling time'], format='%d.%m.%Y %H:%M').dt.tz_localize('UTC').apply(lambda x: x.isoformat())
df["weight"] = 1

In [4]:
gdf = gpd.read_file("lake_geometry.json")
gdf['geometry'] = gdf['geometry'].buffer(0.001)

ERROR 1: PROJ: proj_create_from_database: Open of /home/runnalja/anaconda3/envs/airflow/share/proj failed
/tmp/ipykernel_120443/3964138788.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].buffer(0.001)


In [5]:
results = {}
failed = {}
for index, row in df.iterrows():
    point = Point(row["longitude"], row["latitude"])
    is_inside = gdf.contains(point)
    if is_inside.any():
        polygons_containing_point = gdf[is_inside]
        if len(polygons_containing_point) != 1:
            print("Overlapping polygons")
            print(polygons_containing_point)
        else:
            lake = polygons_containing_point.iloc[0]["key"]
            if lake in results:
                results[lake].append(index)
            else:
                results[lake] = [index]
    else:
        if row["Water body"] not in failed:
            print(row["Water body"])
            failed[row["Water body"]] = 0
        else:
            failed[row["Water body"]] = failed[row["Water body"]] + 1

Aarbergerweiher
Ägelsee
Arnensee
Bachsee (Bachalpsee)
Bachsee (unterer)
Baggersee Hunzigen
Baggersee Kiesen
Baggersee Meienried
Baggersee Münsigen
Bannalpsee
Biaufond
Biessenhofer Weiher Mitte
Birkehofweiher
Blausee
Bleienbacher Torfsee
Blüemlisalpsee
Büeltigenseelein
Cul des Prés
Egolzwilersee
Elsigsee
Engstlensee
Fabrikweiher F.+K. Jenny AG
Flueseeli
Fräschelsweiher
Gadenlauisee
Gantrischseeli
Göscheneralpsee
Grosser Lauenensee
Grossweiher
Gütschweiher
Hagelseewli
Hinterburgseeli
Hinterstockensee
Iffigsee
Junzlenseelein: west
Kiesgrube Heimberg-S
Kwatt Weiher
La Marnière
La Noz
Lac de Nervaux
Lac de Ter
Lac des Taillers
Lac Vert
Lauenensee: kleiner
Les Chaufours
Lütscheren: westlich
Meienfallseeli
Melchsee
Muemethaler Weier
Muggeseeli
Mühliguetweiher
Näelser: Obersee
Oberalpsee
Oberer Chatzensee
Oberstockensee
Remersee
Rezligletschersee
Sängeliweiher
Schiffenensee
Schwarzsee
Seebergsee
Seebodensee
Seelisbergsee
Sgistalsee
Siselenweiher
Spittelmattesee
Steinibühlweiher
Sulssewli
Sulss

In [6]:
df["value"] = df["Wassertemperatur / T-Wasser [°C]"]
df_t = df[["time", "depth", "latitude", "longitude", "value", "weight"]]

for lake in results.keys():
    df_s = df_t.iloc[results[lake]].dropna(subset=['value'])
    os.makedirs('../observations/{}'.format(lake), exist_ok=True)
    df_s.to_csv('../observations/{}/temperature.csv'.format(lake), index=False)
    
df["value"] = df["el_LF / Elektrische Leitfähigkeit [µS/cm]"]
df_t = df[["time", "depth", "latitude", "longitude", "value", "weight"]]

for lake in results.keys():
    df_s = df_t.iloc[results[lake]].dropna(subset=['value'])
    os.makedirs('../observations/{}'.format(lake), exist_ok=True)
    df_s.to_csv('../observations/{}/conductivity.csv'.format(lake), index=False)

### Secchi Depth

In [7]:
df = pd.read_csv("Secchi.csv", encoding='latin1', sep=";")
lat, lng = ch1903_to_latlng(np.array(df["X [m]"]),np.array(df["Y [m]"]))
df["latitude"] = lat
df["longitude"] = lng
df["depth"] = None
df['time'] = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M').dt.tz_localize('UTC').apply(lambda x: x.isoformat())
df["value"] = df["Secchi depth [m]"]
df["weight"] = 1

In [8]:
results = {}
failed = {}
for index, row in df.iterrows():
    point = Point(row["longitude"], row["latitude"])
    is_inside = gdf.contains(point)
    if is_inside.any():
        polygons_containing_point = gdf[is_inside]
        if len(polygons_containing_point) != 1:
            print("Overlapping polygons")
            print(polygons_containing_point)
        else:
            lake = polygons_containing_point.iloc[0]["key"]
            if lake in results:
                results[lake].append(index)
            else:
                results[lake] = [index]
    else:
        if row["Lake"] not in failed:
            print(row["Lake"])
            failed[row["Lake"]] = 0
        else:
            failed[row["Lake"]] = failed[row["Lake"]] + 1

SÃ¤ngeliweiher
oberer Katzensee
La Noz
Schiffenensee
Biessenhofer Weiher Mitte
Kwatt Weiher
Atzenholzweiher
Bildweiher
GrÃ¤ppelensee
Eselschwanzweiher
Schlossweiher
Schwendisee
Stadtweiher Wil
Voralpsee
Wenigerweiher
Wichenstein Gross
Wichenstein Klein


In [9]:
df_t = df[["time", "depth", "latitude", "longitude", "value", "weight"]]

for lake in results.keys():
    df_s = df_t.iloc[results[lake]].dropna(subset=['value'])
    os.makedirs('../observations/{}'.format(lake), exist_ok=True)
    df_s.to_csv('../observations/{}/secchi.csv'.format(lake), index=False)

### Ice

In [10]:
df = pd.read_csv("Ice.csv", encoding='latin1', sep=";")
lat, lng = ch1903_to_latlng(np.array(df["X [m]"]),np.array(df["Y [m]"]))
df["latitude"] = lat
df["longitude"] = lng
df["depth"] = None
df['time'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y %H:%M').dt.tz_localize('UTC').apply(lambda x: x.isoformat())
df["value"] = df["Ice thickness [m]"]
df["weight"] = 1

In [11]:
results = {}
failed = {}
for index, row in df.iterrows():
    point = Point(row["longitude"], row["latitude"])
    is_inside = gdf.contains(point)
    if is_inside.any():
        polygons_containing_point = gdf[is_inside]
        if len(polygons_containing_point) != 1:
            print("Overlapping polygons")
            print(polygons_containing_point)
        else:
            lake = polygons_containing_point.iloc[0]["key"]
            if lake in results:
                results[lake].append(index)
            else:
                results[lake] = [index]
    else:
        if row["Lake"] not in failed:
            print(row["Lake"])
            failed[row["Lake"]] = 0
        else:
            failed[row["Lake"]] = failed[row["Lake"]] + 1

In [12]:
df_t = df[["time", "depth", "latitude", "longitude", "value", "weight"]]

for lake in results.keys():
    df_s = df_t.iloc[results[lake]].dropna(subset=['value'])
    os.makedirs('../observations/{}'.format(lake), exist_ok=True)
    df_s.to_csv('../observations/{}/ice.csv'.format(lake), index=False)